In [ ]:
class_list = ["수치해석", "헬스케어", "딥러닝"]
student1 = []
def add_class1(class_name):
    global student1
    student1.append(class_name)

add_class1(class_list[0])
add_class1(class_list[1])
student1

In [ ]:
student2 = []
def add_class2(class_name):
    global student2
    student2.append(class_name)
add_class2(class_list[0])
add_class2(class_list[2])
student2

In [ ]:
class Student():
    def __init__(self):
        self.class_ = ????

    def add_class(self, ????):
        self.class_????

student1 = Student()

student1.add_class(class_list[0])
student1.add_class(class_list[1])

student1.class_

In [ ]:
student2 = Student()

student2.add_class(class_list[0])
student2.add_class(class_list[2])

student2.class_

In [ ]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        
    def print_info(self):
        print(f"name: {self.name}\nage: {self.age}")
        
class Student(Person):
    def __init__(self, name, age, major, semester):
        super().__init__(name, age)
        self.major = major
        self.semester = semester
        self.set_info()
        self.class_ = []
        
    def set_info(self):
        self.grade = self.semester // 2
        
    def add_class(self, class_name):
        self.class_.append(class_name)
        
    def print_info(self):
        super().print_info()
        print(f"major: {self.major}\nsemester: {self.semester}\ngrade: {self.grade}")
        
    def print_class_info(self):
        print(f"Take {len(self.class_)} class(es)")
        for cidx, cname in enumerate(self.class_):
            print(f"{cidx+1:d}교시 {cname}")
        
        
class Professor(Person):
    def __init__(self, name, age, office):
        super().__init__(name, age)
        self.office = office
        self.lectures = {}
        
    def add_lecture(self, lecture_name, credit):
        self.lectures[lecture_name] = credit
        
    def print_info(self):
        super().print_info()
        print(f"{len(self.lectures)} 수업")
        
    def print_lecture_info(self):
        print(f"{len(self.lectures)} lecture(s)")
        for cidx, (lname, lcredit) in enumerate(self.lectures.items()):
            print(f"{lcredit}학점 {lname}")

In [ ]:
st1 = Student()
st1.add_class(class_list[0])
st1.add_class(class_list[1])
st1.print_class_info(), st1.print_info()

In [ ]:
pf = Professor()
pf.add_lecture(class_list[0],3)
pf.add_lecture(class_list[1],3)
pf.print_lecture_info(), pf.print_info()

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
import os
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

class MNIST(Dataset):
    def __init__(self, data_path="../Dataset/MNIST", train=True, device=None) -> None:
        super().__init__()
        self.train = train
        self.folder = "training" if self.train else "testing"
        self.shuffle = True if self.train else False
        self.data_path = f"{data_path}/{self.folder}"
        self.device = device
        self.load_data()
        
        self.data = self.data.to(self.device)
        self.targets = self.targets.to(self.device)
        print(f"load {len(self.data)} data for {self.folder}")
        
    def load_data(self):
        self.data = torch.empty(0)
        self.targets = torch.empty(0)
        for i in range(10):
            img_path = f"{self.data_path}/{i}/*.png"
            for idx, fname in enumerate(glob(img_path)):
                img = Image.open(fname)
                img_tensor = ToTensor()(img)[None, ...]
                label = torch.tensor([i])#[None, ...]
                self.data = torch.cat((self.data, img_tensor), dim=0)
                self.targets = torch.cat((self.targets, label), dim=0)
                
                if self.train and idx%5000==5000-1:
                    break 
#             print(f"load {len(self.data)} data")
                    
#                 if (not self.train) and idx%10==10-1:
#                     print(f"load {len(self.data)} data")
#                     break 
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, index: int):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img, target = self.data[index], self.targets[index].to(dtype=torch.int64)
        return img, target

In [ ]:
train_dataset = MNIST(train=True, device=device)
valid_dataset = MNIST(train=False, device=device)

In [ ]:
# vis_dataset = MNIST(train=False, device="cuda")

In [ ]:
batch_size = 512
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size = batch_size, shuffle=False)

In [ ]:
import torch.nn as nn
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
model = NeuralNetwork().to("cuda")
print(model)

<a href="https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html">Cross Entropy Loss</a>

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    print(size, num_batches)
    
    train_loss = 0
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         if batch % 100 == 0:
#             loss, current = loss.item(), (batch + 1) * len(X)
#             print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    return train_loss

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    model.eval()
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct, test_loss

In [ ]:
learning_rate = 0.001
epochs = 100
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
monitor = 20
corrects = []
train_losses = []
test_losses = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train_loop(train_loader, model, loss_fn, optimizer)
    correct, test_loss = test_loop(valid_loader, model, loss_fn)
    corrects.append(correct)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    if t%monitor==monitor-1:
        plt.plot(train_losses, label="train")
        plt.plot(test_losses, label="test")
        plt.legend()
        plt.show()
        plt.close()
    
print("Done!")

## Save checkpoint option1

In [ ]:
os.mkdir("ckpts1")

In [ ]:
learning_rate = 0.001
epochs = 100
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
monitor = 20
corrects = []
train_losses = []
test_losses = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train_loop(train_loader, model, loss_fn, optimizer)
    correct, test_loss = test_loop(valid_loader, model, loss_fn)
    corrects.append(correct)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    if t%monitor==monitor-1:
        plt.plot(train_losses, label="train")
        plt.plot(test_losses, label="test")
        plt.legend()
        plt.show()
        plt.close()
    torch.save(model????,save_path???)
    break
    
print("Done!")

option2

## Save checkpoint option2

In [ ]:
os.mkdir("ckpts2")

In [ ]:
learning_rate = 0.001
epochs = 100
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
monitor = 20
corrects = []
train_losses = []
test_losses = []
min_loss = torch.tensor(float('inf')).cuda()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train_loop(train_loader, model, loss_fn, optimizer)
    correct, test_loss = test_loop(valid_loader, model, loss_fn)
    corrects.append(correct)
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    if t%monitor==monitor-1:
        plt.plot(train_losses, label="train")
        plt.plot(test_losses, label="test")
        plt.legend()
        plt.show()
        plt.close()
    if ????:
        ????
        print("save")
        torch.save(model????,save_path???)
    
print("Done!")

## Visualize

In [ ]:
eval_model = NeuralNetwork().to(device)
eval_model.load model???(????)

In [ ]:
eval_model.eval()
vis_loader = DataLoader(vis_dataset, batch_size = batch_size, shuffle=False)
with torch.no_grad():
    for img, target in vis_loader:
        break
    out = eval_model(img)

In [ ]:
plt.figure(figsize=(13,13))
for i in range(100):
    plt.subplot(10,10,i+1)
    plt.imshow(img[i].permute(1,2,0).cpu().detach())
    plt.title(f"{target[i].item()},{torch.argmax(out[i]).item()}")
    plt.tight_layout()
    plt.axis("off")
plt.show()
plt.close()